In [4]:
import numpy as np 
import pandas as pd 

In [70]:
train = pd.read_csv("../datasets/real_or_not/train.csv")
test = pd.read_csv("../datasets/real_or_not/test.csv")
submission = pd.read_csv("../datasets/real_or_not/sample_submission.csv")

In [71]:
train.head(5)

id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1

In [72]:
test.head(5)

id keyword location                                               text
0   0     NaN      NaN                 Just happened a terrible car crash
1   2     NaN      NaN  Heard about #earthquake is different cities, s...
2   3     NaN      NaN  there is a forest fire at spot pond, geese are...
3   9     NaN      NaN           Apocalypse lighting. #Spokane #wildfires
4  11     NaN      NaN      Typhoon Soudelor kills 28 in China and Taiwan

In [73]:
submission.head(5)

id  target
0   0       0
1   2       0
2   3       0
3   9       0
4  11       0

In [74]:
train = train[['text', 'target']]
test = test[['text']]

train.columns = ['tweet', 'class']
test.columns = ['tweet']

In [75]:
import re

hashtags = re.compile(r"^#\S+|\s#\S+")
mentions = re.compile(r"^@\S+|\s@\S+")
urls1 = r"(?:\@|https?\://)\S+"
urls2 = r"(?:\@|http?\://)\S+"
urls_pat = r'|'.join((urls1, urls2))
urls = re.compile(urls_pat)

def process_text(text):
  text = hashtags.sub(' hashtag', text)
  text = mentions.sub(' entity', text)
  return text.strip().lower()
  
def match_expr(pattern, string):
  return not pattern.search(string) == None

def get_data_wo_urls(dataset):
    link_with_urls = dataset.tweet.apply(lambda x: match_expr(urls, x))
    return dataset[[not e for e in link_with_urls]]

In [76]:
train.tweet = train.tweet.apply(process_text)
train.head(4)

tweet  class
0  our deeds are the reason of this hashtag may a...      1
1             forest fire near la ronge sask. canada      1
2  all residents asked to 'shelter in place' are ...      1
3  13,000 people receive hashtag evacuation order...      1

In [77]:
from sklearn.model_selection import train_test_split
TRAIN_SIZE = 0.75
dataset_count = len(train)

df_train, df_test = train_test_split(train, test_size=1-TRAIN_SIZE, random_state=42)
# df_train_val, df_test = train_test_split(dataset, test_size=1-TRAIN_SIZE-VAL_SIZE, random_state=42)
# df_train, df_val = train_test_split(df_train_val, test_size=VAL_SIZE / (VAL_SIZE + TRAIN_SIZE), random_state=42)

print("TRAIN size:", len(df_train))
# print("VAL size:", len(df_val))
print("TEST size:", len(df_test))

TRAIN size: 5709
TEST size: 1904


In [78]:
df_train = get_data_wo_urls(df_train)
df_train.head(4)

tweet  class
5151  entity entity this makes sense. paper beats ro...      0
3443  well as i was chaning an ipad screen it fuckin...      0
7164  the war on drugs has turned the u.s. into a wa...      1
7037  obama declares disaster for typhoon-devastated...      1

In [79]:
df_test.head(4)

tweet  class
2644  so you have a new weapon that can cause un-ima...      1
2227  the f$&amp;@ing things i do for hashtag just g...      0
5448  dt entity rt entity ûïthe col police can catc...      1
132   aftershock back to school kick off was great. ...      0

In [80]:
import time
import os
from joblib import Parallel, delayed

train_len = len(df_train.index)
texts_train = df_train['tweet'].tolist()
classes_train = df_train['class'].tolist()

test_len = len(df_test.index)
texts_test = df_test['tweet'].tolist()
classes_test = df_test['class'].tolist()

In [81]:
path = os.getcwd()
train_path = os.path.join(path, "..", "datasets", 'bert_real_or_not', 'train', '')
test_path = os.path.join(path, "..", "datasets", 'bert_real_or_not', 'test', '')

def write_txt(text_, class_, i, is_train):
     if class_ == 0:
         class_ = 'fal'
     else:
         class_ = 'tru'
     if is_train == 0:
         cur_path = test_path
     else:
         cur_path = train_path
     with open(os.path.join(cur_path, class_, f'{i}_{class_}.txt'), 'w', encoding='utf-8') as f:
         f.write(text_)

threads=4
Parallel(n_jobs=threads)(delayed(write_txt)(texts_train[i], classes_train[i], i, 1) for i in range(train_len))
Parallel(n_jobs=threads)(delayed(write_txt)(texts_test[i], classes_test[i], i, 0) for i in range(test_len))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [1]:
import ktrain
from ktrain import text
import os

using Keras version: 2.2.4-tf


In [7]:
train = pd.read_csv("../datasets/real_or_not/train.csv")
test = pd.read_csv("../datasets/real_or_not/test.csv")
submission = pd.read_csv("../datasets/real_or_not/sample_submission.csv")

test = test[['text']]
test.columns = ['tweet']

In [8]:
data_to_predict=test['tweet'].tolist()
data_to_predict

['Just happened a terrible car crash',
 'Heard about #earthquake is different cities, stay safe everyone.',
 'there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all',
 'Apocalypse lighting. #Spokane #wildfires',
 'Typhoon Soudelor kills 28 in China and Taiwan',
 "We're shaking...It's an earthquake",
 "They'd probably still show more life than Arsenal did yesterday, eh? EH?",
 'Hey! How are you?',
 'What a nice hat?',
 'Fuck off!',
 "No I don't like cold!",
 "NOOOOOOOOO! Don't do that!",
 "No don't tell me that!",
 'What if?!',
 'Awesome!',
 "Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham's Wholesale Market http://t.co/irWqCEZWEU",
 '@sunkxssedharry will you wear shorts for race ablaze ?',
 '#PreviouslyOnDoyinTv: Toke Makinwa\x89Ûªs marriage crisis sets Nigerian Twitter ablaze... http://t.co/CMghxBa2XI',
 'Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS 

In [9]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' #To ignore GPU

In [10]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder("../datasets/bert_real_or_not",
                                                                        maxlen=500,
                                                                        preprocess_mode="bert",
                                                                        classes=["tru", "fal"])

detected encoding: utf-8
preprocessing train...
language: en


preprocessing test...
language: en


In [11]:
learner = ktrain.get_learner(text.text_classifier("bert", (x_train, y_train), preproc=preproc),
                            train_data = (x_train, y_train),
                            val_data = (x_test, y_test),
                            batch_size = 6)

Is Multi-Label? False
maxlen is 500
done.


In [12]:
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 2699 samples, validate on 1904 samples
2699/2699 [==============================] - 6480s 2s/sample - loss: 0.4988 - accuracy: 0.7758 - val_loss: 0.4162 - val_accuracy: 0.8167


In [13]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictions = predictor.predict(data_to_predict)

In [14]:
predictions

['tru',
 'tru',
 'tru',
 'tru',
 'tru',
 'tru',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'tru',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'tru',
 'fal',
 'tru',
 'fal',
 'fal',
 'fal',
 'tru',
 'fal',
 'fal',
 'tru',
 'tru',
 'tru',
 'fal',
 'tru',
 'fal',
 'fal',
 'tru',
 'fal',
 'tru',
 'tru',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'tru',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'tru',
 'tru',
 'tru',
 'tru',
 'tru',
 'tru',
 'fal',
 'tru',
 'tru',
 'tru',
 'fal',
 'tru',
 'tru',
 'tru',
 'tru',
 'tru',
 'fal',
 'fal',
 'fal',
 'tru',
 'fal',
 'fal',
 'fal',
 'tru',
 'tru',
 'fal',
 'fal',
 'fal',
 'tru',
 'tru',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'tru',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'fal',
 'tru',
 'fal',
 'fal',
 'tru',
 'tru',
 'fal',
 'fal',
 'fal',
 'tru',
 'tru',
 'fal',
 'tru',
 'tru',
 'fal',
 'tru',
 'tru',
 'tru',
 'tru',
 'fal',


In [18]:
preds = []
for item in predictions:
    if item == 'tru':
        preds.append(1)
    else:
        preds.append(0)
preds

[1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [20]:
submission['target'] = preds
submission.to_csv('submission_ktrain.csv', index=False)

In [21]:
len(preds)

3263

In [22]:
len(submission)

3263